In [1]:
import numpy as np
import pandas as pd
import glob

## Table 2 in submission [done]

In [13]:
# For individual task
root_paths = ['/data/UKBB/SSL/reports/permutation/*csv', '/data/UKBB/SSL/reports/aot/*csv', 
              '/data/UKBB/SSL/reports/scale/*csv', '/data/UKBB/SSL/reports/time_w/*csv']
              
master_file_list = []
for root_path in root_paths:
    current_list = glob.glob(root_path)
    master_file_list.extend(current_list)

In [14]:
# get the ones with error bar
master_df = []
for file_path in master_file_list:
    file_name = file_path.split('/')[-1]
    dataset = file_name.split('_')[0]
    
    subject_num = file_name.split('_')[-1]
    subject_num = int(subject_num.split('.')[0])
    model = '_'.join((file_name.split('_')[1:-1]))
    task = file_path.split('/')[-2]

    # epoch_count = file_name.split('_')[-1][:2]
    
    current_df = pd.read_csv(file_path)
    
    current_df['id'] = current_df.index 
    current_df = pd.melt(current_df, id_vars=['id'])
    current_df.drop(['id'], inplace=True, axis=1)

    current_df['dataset'] = dataset
    current_df['model'] = model
    # print(file_path, model, subject_num)
    
    current_df['subject_num'] = subject_num  
    current_df['task'] = task
    
    if len(master_df) == 0:
        master_df = current_df
    else:
        master_df = pd.concat([master_df, current_df])
master_df = master_df.rename(columns={"variable": "metric"}) 
#master_df['epoch'] = master_df['epoch'].astype(int)

In [15]:
master_df = master_df[['task', 'model', 'metric', 'value', 'dataset']]

In [16]:
master_df['task']

0     permutation
1     permutation
2     permutation
3     permutation
4     permutation
         ...     
15         time_w
16         time_w
17         time_w
18         time_w
19         time_w
Name: task, Length: 4784, dtype: object

In [17]:
master_df.head()

,task,model,metric,value,dataset
0,permutation,flip_net_ft_100_postR60_1500_-1,precision,0.861416,rowlands
1,permutation,flip_net_ft_100_postR60_1500_-1,precision,0.811094,rowlands
2,permutation,flip_net_ft_100_postR60_1500_-1,precision,0.924670,rowlands
3,permutation,flip_net_ft_100_postR60_1500_-1,precision,0.831987,rowlands
4,permutation,flip_net_ft_100_postR60_1500_-1,precision,0.884477,rowlands


In [18]:
filter_master_df = master_df[(master_df['metric'] == 'f1') | (master_df['metric'] == 'kappa')]

In [19]:
key=['dataset', 'metric', 'task']
ok = filter_master_df.groupby( key).describe()

In [22]:
ok = ok.reset_index(['dataset', 'metric' , 'task' ])

In [24]:
ok.head()

dataset metric         task value                                          \
                              count      mean       std       min       25%   
0     adl     f1          aot   7.0  0.754390  0.157191  0.467779  0.679454   
1     adl     f1  permutation   7.0  0.809835  0.096647  0.660000  0.756569   
2     adl     f1        scale   7.0  0.410983  0.185836  0.105201  0.304262   
3     adl     f1       time_w   7.0  0.764831  0.116905  0.573044  0.702677   
4     adl  kappa          aot   7.0  0.730352  0.209764  0.388446  0.591625   

                                 
        50%       75%       max  
0  0.820000  0.864641  0.904762  
1  0.792589  0.880996  0.941127  
2  0.429962  0.556292  0.620612  
3  0.800578  0.825523  0.923791  
4  0.814683  0.878970  0.968145

In [ ]:
# read out single task table 

In [35]:
np.unique(ok.task)

array(['aot', 'permutation', 'scale', 'time_w'], dtype=object)

In [25]:
np.unique(ok.dataset)

array(['adl', 'capture24', 'oppo', 'pamap', 'realworld', 'rowlands',
       'wisdm'], dtype=object)

In [37]:
def get_row_str(my_df, task_name):
    task_df = my_df[my_df.task == task_name]
    data_set_order = ['capture24', 'rowlands', 'wisdm', 'realworld', 'oppo', 'pamap', 'adl']
    
    res_str = ""
    for data_set in data_set_order:
        mean= task_df[task_df['dataset'] == data_set].iloc[0].value['mean']
        std =task_df[task_df['dataset'] == data_set].iloc[0].value['std']
        if len(res_str) == 0:
            res_str = r'%.3f $ \pm $ %.3f' % (mean, std)
        else:
            res_str = res_str + r' & %.3f $ \pm $ %.3f' % (mean, std)
            
    print(res_str.replace('0.', '.'))

In [38]:
get_row_str(ok, 'aot')

.671 $ \pm $ .094 & .565 $ \pm $ .120 & .767 $ \pm $ .124 & .750 $ \pm $ .084 & .582 $ \pm $ .054 & .715 $ \pm $ .036 & .754 $ \pm $ .157


In [39]:
get_row_str(ok, 'scale')

.708 $ \pm $ .094 & .692 $ \pm $ .095 & .634 $ \pm $ .123 & .740 $ \pm $ .089 & .293 $ \pm $ .146 & .607 $ \pm $ .083 & .411 $ \pm $ .186


In [40]:
get_row_str(ok, 'permutation')

.721 $ \pm $ .093 & .783 $ \pm $ .099 & .778 $ \pm $ .109 & .766 $ \pm $ .063 & .588 $ \pm $ .076 & .750 $ \pm $ .057 & .810 $ \pm $ .097


In [41]:
get_row_str(ok, 'time_w')

.715 $ \pm $ .093 & .776 $ \pm $ .110 & .767 $ \pm $ .120 & .772 $ \pm $ .073 & .584 $ \pm $ .064 & .737 $ \pm $ .079 & .765 $ \pm $ .117


## Table 3 submission [done]

In [2]:
root_path = '/data/UKBB/SSL/reports/subject_wise/*'
exp_list = glob.glob(root_path)


master_file_list = []
for my_path in exp_list:
    current_list = glob.glob(my_path+'/*csv')
    master_file_list.extend(current_list)

In [65]:
# For individual task
# root_paths = ['/data/UKBB/SSL/reports/aTrue_pTrue_sFalse_tFalse/*csv', '/data/UKBB/SSL/reports/aFalse_pTrue_sTrue_tFalse/*csv', 
#               '/data/UKBB/SSL/reports/aFalse_pTrue_sTrue_tTrue/*csv', '/data/UKBB/SSL/reports/aFalse_pTrue_sFalse_tTrue/*csv',
#              '/data/UKBB/SSL/reports/aTrue_pTrue_sTrue_tFalse/*csv', '/data/UKBB/SSL/reports/aTrue_pTrue_sTrue_tTrue/*csv',
#              '/data/UKBB/SSL/reports/aFalse_pFalse_sTrue_tTrue/*csv', '/data/UKBB/SSL/reports/aTrue_pFalse_sTrue_tFalse/*csv',
#              '/data/UKBB/SSL/reports/aTrue_pFalse_sFalse_tTrue/*csv',  '/data/UKBB/SSL/reports/aTrue_pTrue_sFalse_tTrue/*csv']
              
# master_file_list = []
# for root_path in root_paths:
#     current_list = glob.glob(root_path)
#     master_file_list.extend(current_list)

In [66]:
mtl_dict = {
    'aTrue_pTrue_sFalse_tFalse': 'P+AoT',
    'aTrue_pFalse_sTrue_tFalse': 'AoT+S',
    'aTrue_pFalse_sFalse_tTrue': 'AoT+TW',
    'aFalse_pTrue_sTrue_tFalse': 'P+S',
    'aFalse_pTrue_sTrue_tTrue': 'P+S+Tw',
    'aFalse_pTrue_sFalse_tTrue': 'P+Tw',
    'aTrue_pTrue_sTrue_tTrue': 'P+AoT+S+TW',
    'aTrue_pTrue_sTrue_tFalse': 'P+AoT+S',
    'aFalse_pFalse_sTrue_tTrue': 'S+TW',
    'aTrue_pTrue_sFalse_tTrue': 'AoT+P+TW',
    'aTrue_pFalse_sTrue_tTrue': 'AoT+S+TW',
    'aot': 'AoT',
    'permutation': 'p',
    'time_w': 'TW',
    'scale': 'scale'
}

In [67]:
# get the ones with error bar
master_df = []
for file_path in master_file_list:
    file_name = file_path.split('/')[-1]
    dataset = file_name.split('_')[0]
    
    subject_num = file_name.split('_')[-1]
    subject_num = int(subject_num.split('.')[0])
    task = file_path.split('/')[-2]
    mtl_task = mtl_dict[file_path.split('/')[-2]]
    
    # epoch_count = file_name.split('_')[-1][:2]
    
    current_df = pd.read_csv(file_path)
    
    current_df['id'] = current_df.index 
    current_df = pd.melt(current_df, id_vars=['id'])
    current_df.drop(['id'], inplace=True, axis=1)

    current_df['dataset'] = dataset
    # print(file_path, model, subject_num)
    
    current_df['subject_num'] = subject_num  
    current_df['task'] = task
    current_df['mtl_task'] = mtl_task


    if len(master_df) == 0:
        master_df = current_df
    else:
        master_df = pd.concat([master_df, current_df])
master_df = master_df.rename(columns={"variable": "metric"}) 

In [68]:
master_df = master_df[['task', 'mtl_task', 'metric', 'value', 'dataset']]
filter_master_df = master_df[(master_df['metric'] == 'f1') | (master_df['metric'] == 'kappa')]

In [69]:
key=['dataset', 'metric', 'task']
ok = filter_master_df.groupby( key).describe()

In [70]:
ok

value                      \
                                            count      mean       std   
dataset   metric task                                                   
capture24 f1     aFalse_pFalse_sTrue_tTrue  155.0  0.707652  0.094510   
                 aFalse_pTrue_sFalse_tTrue  155.0  0.714490  0.094406   
                 aFalse_pTrue_sTrue_tFalse  155.0  0.696643  0.096855   
                 aFalse_pTrue_sTrue_tTrue   155.0  0.710223  0.093890   
                 aTrue_pFalse_sFalse_tTrue  155.0  0.718973  0.093533   
...                                           ...       ...       ...   
rowlands  kappa  permutation                 55.0  0.874980  0.072122   
                 scale                       55.0  0.801078  0.080509   
                 time_w                      55.0  0.861312  0.089002   
wisdm     f1     aTrue_pTrue_sFalse_tTrue    50.0  0.576952  0.090907   
          kappa  aTrue_pTrue_sFalse_tTrue    50.0  0.608217  0.099776   

                                                                          \
                                                 min       25%       50%   
dataset   metric task                                                      
capture24 f1     aFalse_pFalse_sTrue_tTrue  0.378442  0.633251  0.724155   
                 aFalse_pTrue_sFalse_tTrue  0.399576  0.644024  0.734623   
                 aFalse_pTrue_sTrue_tFalse  0.387989  0.617306  0.714469   
                 aFalse_pTrue_sTrue_tTrue   0.387488  0.646414  0.722340   
                 aTrue_pFalse_sFalse_tTrue  0.409476  0.648563  0.734550   
...                                              ...       ...       ...   
rowlands  kappa  permutation                0.691126  0.830284  0.883346   
                 scale                      0.641608  0.751697  0.817109   
                 time_w                     0.624836  0.794210  0.874574   
wisdm     f1     aTrue_pTrue_sFalse_tTrue   0.175242  0.564901  0.588655   
          kappa  aTrue_pTrue_sFalse_tTrue   0.159933  0.597986  0.630623   

                                                                
                                                 75%       max  
dataset   metric task                                           
capture24 f1     aFalse_pFalse_sTrue_tTrue  0.778304  0.893114  
                 aFalse_pTrue_sFalse_tTrue  0.783370  0.900776  
                 aFalse_pTrue_sTrue_tFalse  0.773808  0.877638  
                 aFalse_pTrue_sTrue_tTrue   0.781666  0.887968  
                 aTrue_pFalse_sFalse_tTrue  0.790456  0.896979  
...                                              ...       ...  
rowlands  kappa  permutation                0.928717  0.990690  
                 scale                      0.857155  0.941710  
                 time_w                     0.931544  0.988371  
wisdm     f1     aTrue_pTrue_sFalse_tTrue   0.624274  0.672541  
          kappa  aTrue_pTrue_sFalse_tTrue   0.651246  0.711858  

[62 rows x 8 columns]

In [71]:
ok = ok.reset_index(['dataset', 'metric' , 'task' ])

In [72]:
ok[ok['dataset'] == 'oppo']

Empty DataFrame
Columns: [(dataset, ), (metric, ), (task, ), (value, count), (value, mean), (value, std), (value, min), (value, 25%), (value, 50%), (value, 75%), (value, max)]
Index: []

In [73]:
def get_row_str(my_df, task_name):
    task_df = my_df[my_df.task == task_name]

    mean= task_df[task_df['dataset'] == 'capture24'].iloc[0].value['mean']
    std =task_df[task_df['dataset'] == 'capture24'].iloc[0].value['std']
    mean_row = task_df[task_df['dataset'] == 'rowlands'].iloc[0].value['mean']
    std_row =task_df[task_df['dataset'] == 'rowlands'].iloc[0].value['std']

    tmp_str = r'%.3f $ \pm $ %.3f & %.3f $ \pm $  %.3f' % (mean, std, mean_row, std_row)
    print(tmp_str)

In [74]:
task_name  = 'aTrue_pFalse_sTrue_tFalse'
get_row_str(ok, task_name)


0.702 $ \pm $ 0.094 & 0.686 $ \pm $  0.094


In [75]:
task_name  = 'aTrue_pTrue_sFalse_tFalse'
get_row_str(ok, task_name)



0.718 $ \pm $ 0.092 & 0.781 $ \pm $  0.101


In [76]:
task_name  = 'aTrue_pFalse_sFalse_tTrue'
get_row_str(ok, task_name)

0.719 $ \pm $ 0.094 & 0.762 $ \pm $  0.102


In [77]:
task_name  = 'aFalse_pFalse_sTrue_tTrue'
get_row_str(ok, task_name)

0.708 $ \pm $ 0.095 & 0.731 $ \pm $  0.106


In [78]:
task_name  = 'aFalse_pTrue_sFalse_tTrue'
get_row_str(ok, task_name)

0.714 $ \pm $ 0.094 & 0.755 $ \pm $  0.103


In [79]:
task_name  = 'aFalse_pTrue_sFalse_tTrue'
get_row_str(ok, task_name)

0.714 $ \pm $ 0.094 & 0.755 $ \pm $  0.103


In [80]:
task_name  = 'aTrue_pFalse_sFalse_tTrue'
get_row_str(ok, task_name)


0.719 $ \pm $ 0.094 & 0.762 $ \pm $  0.102


In [81]:
task_name  = 'aFalse_pTrue_sTrue_tTrue'
get_row_str(ok, task_name)



0.710 $ \pm $ 0.094 & 0.757 $ \pm $  0.099


In [82]:
task_name  = 'aTrue_pTrue_sFalse_tTrue'
get_row_str(ok, task_name)




0.718 $ \pm $ 0.095 & 0.770 $ \pm $  0.102


In [83]:
task_name  = 'aTrue_pFalse_sTrue_tTrue'
get_row_str(ok, task_name)


0.702 $ \pm $ 0.095 & 0.779 $ \pm $  0.099


In [84]:
task_name  = 'aTrue_pTrue_sTrue_tFalse'
get_row_str(ok, task_name)

0.705 $ \pm $ 0.095 & 0.679 $ \pm $  0.106


In [85]:
task_name  = 'aTrue_pTrue_sTrue_tTrue'
get_row_str(ok, task_name)

0.709 $ \pm $ 0.091 & 0.775 $ \pm $  0.103


In [86]:
task_name  = 'aot'
get_row_str(ok, task_name)

0.671 $ \pm $ 0.094 & 0.537 $ \pm $  0.077


In [87]:
task_name  = 'permutation'
get_row_str(ok, task_name)

0.721 $ \pm $ 0.093 & 0.775 $ \pm $  0.098


In [88]:
task_name  = 'scale'
get_row_str(ok, task_name)

0.708 $ \pm $ 0.094 & 0.683 $ \pm $  0.092


In [89]:
task_name  = 'time_w'
get_row_str(ok, task_name)

0.715 $ \pm $ 0.093 & 0.767 $ \pm $  0.111


In [90]:
task_name  = 'aTrue_pTrue_sTrue_tTrue'
get_row_str(ok, task_name)

0.709 $ \pm $ 0.091 & 0.775 $ \pm $  0.103


In [19]:
with pd.option_context('display.float_format', '{:0.3f}'.format):
    print(ok)

                                             value                            \
                                             count  mean   std    min    25%   
dataset   metric task                                                          
adl       f1     aFalse_pTrue_sFalse_tTrue   7.000 0.841 0.072  0.740  0.800   
                 aFalse_pTrue_sTrue_tFalse   7.000 0.544 0.123  0.316  0.514   
                 aFalse_pTrue_sTrue_tTrue    7.000 0.801 0.089  0.674  0.749   
          kappa  aFalse_pTrue_sFalse_tTrue   7.000 0.851 0.078  0.739  0.803   
                 aFalse_pTrue_sTrue_tFalse   7.000 0.517 0.131  0.283  0.453   
                 aFalse_pTrue_sTrue_tTrue    7.000 0.845 0.094  0.640  0.851   
capture24 f1     aFalse_pFalse_sTrue_tTrue 155.000 0.505 0.091  0.287  0.436   
                 aFalse_pTrue_sFalse_tTrue 155.000 0.518 0.093  0.303  0.445   
                 aFalse_pTrue_sTrue_tFalse 155.000 0.480 0.086  0.286  0.413   
                 aFalse_pTrue_sTrue_tTru